# Target Aquisition and Calculations

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
def loadImage(path):
    image = cv2.imread(path)
    return image

In [ ]:
BGR = 0
HSV = 1
BW  = 2
def displayImage(input_image, type):
    if type == 0: # BGR
        plt.imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    elif type == 1: # HSV
        plt.imshow(cv2.cvtColor(input_image, cv2.COLOR_HSV2RGB))
    else: # B&W
        plt.imshow(input_image)
    plt.show()

In [ ]:
def performRange(input_image, lower, upper):
    return cv2.inRange(input_image, lower, upper)

In [ ]:
image = loadImage("../2016_RealFullField/20.jpg")
displayImage(image, BGR)

## Perform an inRange operation using HSV
We are going to convert our BGR image to the [HSV color space](https://en.wikipedia.org/wiki/HSL_and_HSV) and then select just the image pixels that fall in-between two colors. Since we are looking for a specific reflected greenish color we can use a fairly narrow Hue range (85-95 in our case) and accomodate a large range for saturation and value/brightness (100-255 in our case). This is usually harder to achieve with BGR images since all three channels change together with the amount of light in the image.

Blurring the image is often used to remove small features we're not interested in (noise). We'll compare our result with and without blurring the image first.
 
### HSV lower and upper limits
Select a lower and upper HSV bound color for the **inRange** operation and display these colors next to the to the target image below.

In [ ]:
hsv_lower = (85, 100, 100)
hsv_upper = (105, 255, 255)

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(np.uint8([[hsv_lower]*100]*100), cv2.COLOR_HSV2RGB))
plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(image[:300, 200:500], cv2.COLOR_BGR2RGB))
plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(np.uint8([[hsv_upper]*100]*100), cv2.COLOR_HSV2RGB))
plt.show()

### Perform inRange without blurring

In [ ]:
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
ranged_image = cv2.inRange(hsv_image, hsv_lower, hsv_upper)
displayImage(ranged_image, BW)

### Perform inRange with blurring

In [ ]:
blurred_hsv_image = cv2.blur(hsv_image, (3, 3))
ranged_image = cv2.inRange(blurred_hsv_image, hsv_lower, hsv_upper)
displayImage(ranged_image, BW)

## Morphological Noise Removal
[Morphological Closing](https://en.wikipedia.org/wiki/Closing_%28morphology%29) is dilation followed by erosion and is used for closing small holes. Blurring did a good job removing these holes in our particular scenario, so it is not needed.

In [ ]:
def performClosing(input_image, size=1):
    element = cv2.getStructuringElement(cv2.MORPH_RECT, (size, size))
    output_image = cv2.dilate(input_image, element)
    return cv2.erode(output_image, element)

In [ ]:
closed_image = performClosing(ranged_image, size=2)
displayImage(closed_image, BW)

[Morphological Opening](https://en.wikipedia.org/wiki/Opening_%28morphology%29) is erosion followed by dilation and is used for removing small objects. Our target is so much larger than the noise present that this is probably not needed.

In [ ]:
def performOpening(input_image, size=1):
    element = cv2.getStructuringElement(cv2.MORPH_RECT, (size, size))
    output_image = cv2.erode(input_image, element)
    return cv2.dilate(output_image, element)


In [ ]:
opened_image = performOpening(ranged_image, size=14) # 14 is pretty extreme
displayImage(opened_image, BW)

## Finding Features of Interest
A simple test is to find the bounding box of the largest contour.

In [ ]:
def findTarget(image, mask, overlay=0):
    image = image.copy()
    mask = mask.copy()
    _, contours, _ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # check to see if any contours were found
    if len(contours) > 0:
        # sort the contours and find the largest one
        target = sorted(contours, key = cv2.contourArea, reverse = True)[0]
        
        if overlay == 0:
            # compute the bounding box and draw it
            box = np.int32(cv2.boxPoints(cv2.minAreaRect(target)))
            cv2.drawContours(image, [box], -1, (0, 0, 255), 1)
        else:
            # draw the contour
            cv2.drawContours(image, [target], -1, (0, 0, 255), 2)
    return image

In [ ]:
marked_image = findTarget(image, ranged_image, 1)
displayImage(marked_image, BGR)
    
# zoom in
displayImage(marked_image[150:200, 250:300], BGR)
displayImage(marked_image[150:200, 425:475], BGR)

In [ ]:
def displaySamples(samples):
    for img in ["../2016_RealFullField/{}.jpg".format(i) for i in samples]:
        image = loadImage(img)
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        blurred_hsv_image = cv2.blur(hsv_image, (3, 3))
        ranged_image = cv2.inRange(blurred_hsv_image, hsv_lower, hsv_upper)
        marked_image = findTarget(image, ranged_image, 1)

        print(img)
        displayImage(marked_image, BGR)

### Single Target in FOV

In [ ]:
import itertools
displaySamples(itertools.chain([9], range(13, 17), range(19, 34)))

### Two Targets in FOV

In [ ]:

displaySamples(itertools.chain([0], range(3, 8)))